In [1]:
import torch,torchvision
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
import numpy as np
import pandas as pd
import wandb
import os, json, cv2, random
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor,DefaultTrainer
from detectron2.config import get_cfg
from detectron2.structures import BoxMode
from tqdm import tqdm
import matplotlib.pyplot as plt
from detectron2.utils.visualizer import ColorMode
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
PROJECT_NAME = 'Car-Object-Detection-V11-Learning-Detectron2-V2'

In [2]:
data = pd.read_csv('./data.csv')

In [3]:
data

               image        xmin        ymin        xmax        ymax
0     vid_4_1000.jpg  281.259045  187.035071  327.727931  223.225547
1    vid_4_10000.jpg   15.163531  187.035071  120.329957  236.430180
2    vid_4_10040.jpg  239.192475  176.764801  361.968162  236.430180
3    vid_4_10020.jpg  496.483358  172.363256  630.020260  231.539575
4    vid_4_10060.jpg   16.630970  186.546010  132.558611  238.386422
..               ...         ...         ...         ...         ...
554   vid_4_9860.jpg    0.000000  198.321729   49.235251  236.223284
555   vid_4_9880.jpg  329.876184  156.482351  536.664239  250.497895
556   vid_4_9900.jpg    0.000000  168.295823  141.797524  239.176652
557   vid_4_9960.jpg  487.428988  172.233646  616.917699  228.839864
558   vid_4_9980.jpg  221.558631  182.570434  348.585579  238.192196

[559 rows x 5 columns]

In [4]:
img = cv2.imread('./data/' + info['image'])
xmin,ymin,xmax,ymax = info['xmin'],info['ymin'],info['xmax'],info['ymax']
x = xmin
y = ymin
w = xmax - xmin
h = ymax - ymin
crop = img[y:y+h,x:x+w]
cv2.imwrite(crop,'./crop.png')
cv2.rectangle(img,(x,y),(x+w,y+h),(200,0,0),2)

In [5]:
info = data.iloc[0]

In [6]:
img = cv2.imread('./data/' + info['image'])
xmin,ymin,xmax,ymax = info['xmin'],info['ymin'],info['xmax'],info['ymax']
x = xmin
y = ymin
w = xmax - xmin
h = ymax - ymin
crop = img[y:y+h,x:x+w]
cv2.imwrite(crop,'./crop.png')
cv2.rectangle(img,(x,y),(x+w,y+h),(200,0,0),2)

In [7]:
len(data)

559

In [8]:
data

               image        xmin        ymin        xmax        ymax
0     vid_4_1000.jpg  281.259045  187.035071  327.727931  223.225547
1    vid_4_10000.jpg   15.163531  187.035071  120.329957  236.430180
2    vid_4_10040.jpg  239.192475  176.764801  361.968162  236.430180
3    vid_4_10020.jpg  496.483358  172.363256  630.020260  231.539575
4    vid_4_10060.jpg   16.630970  186.546010  132.558611  238.386422
..               ...         ...         ...         ...         ...
554   vid_4_9860.jpg    0.000000  198.321729   49.235251  236.223284
555   vid_4_9880.jpg  329.876184  156.482351  536.664239  250.497895
556   vid_4_9900.jpg    0.000000  168.295823  141.797524  239.176652
557   vid_4_9960.jpg  487.428988  172.233646  616.917699  228.839864
558   vid_4_9980.jpg  221.558631  182.570434  348.585579  238.192196

[559 rows x 5 columns]

In [9]:
data['xmin'].type

In [10]:
data['xmin'].dtype

dtype('float64')

In [11]:
data['xmin'].astype(int)

0      281
1       15
2      239
3      496
4       16
      ... 
554      0
555    329
556      0
557    487
558    221
Name: xmin, Length: 559, dtype: int64

In [12]:
data['xmin'] = data['xmin'].astype(int)
data['ymin'] = data['ymin'].astype(int)
data['xmax'] = data['xmax'].astype(int)
data['ymax'] = data['ymax'].astype(int)

In [13]:
img = cv2.imread('./data/' + info['image'])
xmin,ymin,xmax,ymax = info['xmin'],info['ymin'],info['xmax'],info['ymax']
x = xmin
y = ymin
w = xmax - xmin
h = ymax - ymin
crop = img[y:y+h,x:x+w]
cv2.imwrite(crop,'./crop.png')
cv2.rectangle(img,(x,y),(x+w,y+h),(200,0,0),2)

In [14]:
info = data.iloc[0]

In [15]:
data['xmin'] = data['xmin'].astype(int)
data['ymin'] = data['ymin'].astype(int)
data['xmax'] = data['xmax'].astype(int)
data['ymax'] = data['ymax'].astype(int)

In [16]:
img = cv2.imread('./data/' + info['image'])
xmin,ymin,xmax,ymax = info['xmin'],info['ymin'],info['xmax'],info['ymax']
x = xmin
y = ymin
w = xmax - xmin
h = ymax - ymin
crop = img[y:y+h,x:x+w]
cv2.imwrite(crop,'./crop.png')
cv2.rectangle(img,(x,y),(x+w,y+h),(200,0,0),2)

In [17]:
img = cv2.imread('./data/' + info['image'])
xmin,ymin,xmax,ymax = info['xmin'],info['ymin'],info['xmax'],info['ymax']
x = xmin
y = ymin
w = xmax - xmin
h = ymax - ymin
crop = img[y:y+h,x:x+w]
cv2.imwrite('./crop.png',crop)
cv2.rectangle(img,(x,y),(x+w,y+h),(200,0,0),2)
cv2.imwrite('./box.png',box)

In [18]:
img = cv2.imread('./data/' + info['image'])
xmin,ymin,xmax,ymax = info['xmin'],info['ymin'],info['xmax'],info['ymax']
x = xmin
y = ymin
w = xmax - xmin
h = ymax - ymin
crop = img[y:y+h,x:x+w]
cv2.imwrite('./crop.png',crop)
cv2.rectangle(img,(x,y),(x+w,y+h),(200,0,0),2)
cv2.imwrite('./box.png',img)

True

In [19]:
info = data.iloc[59]

In [20]:
data['xmin'] = data['xmin'].astype(int)
data['ymin'] = data['ymin'].astype(int)
data['xmax'] = data['xmax'].astype(int)
data['ymax'] = data['ymax'].astype(int)

In [21]:
img = cv2.imread('./data/' + info['image'])
xmin,ymin,xmax,ymax = info['xmin'],info['ymin'],info['xmax'],info['ymax']
x = xmin
y = ymin
w = xmax - xmin
h = ymax - ymin
crop = img[y:y+h,x:x+w]
cv2.imwrite('./crop.png',crop)
cv2.rectangle(img,(x,y),(x+w,y+h),(200,0,0),2)
cv2.imwrite('./box.png',img)

True

In [22]:
info = data.iloc[599]

In [23]:
info = data.iloc[550]

In [24]:
data['xmin'] = data['xmin'].astype(int)
data['ymin'] = data['ymin'].astype(int)
data['xmax'] = data['xmax'].astype(int)
data['ymax'] = data['ymax'].astype(int)

In [25]:
img = cv2.imread('./data/' + info['image'])
xmin,ymin,xmax,ymax = info['xmin'],info['ymin'],info['xmax'],info['ymax']
x = xmin
y = ymin
w = xmax - xmin
h = ymax - ymin
crop = img[y:y+h,x:x+w]
cv2.imwrite('./crop.png',crop)
cv2.rectangle(img,(x,y),(x+w,y+h),(200,0,0),2)
cv2.imwrite('./box.png',img)

True

In [26]:
info = data.iloc[185]

In [27]:
data['xmin'] = data['xmin'].astype(int)
data['ymin'] = data['ymin'].astype(int)
data['xmax'] = data['xmax'].astype(int)
data['ymax'] = data['ymax'].astype(int)

In [28]:
img = cv2.imread('./data/' + info['image'])
xmin,ymin,xmax,ymax = info['xmin'],info['ymin'],info['xmax'],info['ymax']
x = xmin
y = ymin
w = xmax - xmin
h = ymax - ymin
crop = img[y:y+h,x:x+w]
cv2.imwrite('./crop.png',crop)
cv2.rectangle(img,(x,y),(x+w,y+h),(200,0,0),2)
cv2.imwrite('./box.png',img)

True

In [29]:
def load_data():
    records = []
    for i in tqdm(range(len(data))):
        record = {}
        info = data.iloc[i]
        path = './data/' + info['image']
        record['file_name'] = path
        record['height'],record['width'] = cv2.imread(path).shape[:2]
        record['image_id'] = i
        record['annotations'] = [
            {
                'bbox':[info['xmin'],info['ymin'],info['xmax'],info['ymax']],
                'bbox_mode':BoxMode.XYXY_ABS,
                'category_id':0
            }
        ]
        records.append(record)
    np.save(records,'./data.npy')
    return records

In [30]:
labels = ['car']

In [31]:
DatasetCatalog.register('data',lambda : load_data())
MetadataCatalog.get('data').set(thing_classes=labels)
metadata = MetadataCatalog.get('data')

In [32]:
wandb.init(project=PROJECT_NAME,name='baseline')
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(model))
cfg.DATASETS.TRAIN = ('data')
cfg.DATASETS.TEST = ()
cfg.SOLVER.STEPS = []
cfg.SOLVER.MAX_ITER = 500
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(labels)
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(model)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()
predictor = DefaultPredictor(cfg)
cfg.MODEL.WEIGHTS = './output/model_final.pth'
evaluator = COCOEvaluator('data',output_dir='./output/')
val_loader = build_detection_test_loader(cfg,'data')
metrics = inference_on_dataset(predictor.model,val_loader,evaluator)
wandb.log(metrics)
img = cv2.imread('./data/vid_4_12220.jpg')
preds = predictor(img)
v = Visualizer(img[:,:,::-1],metadata=metadata)
v = v.draw_instance_predictions(preds['instances'].to('cpu'))
v = v.get_image()[:,:,::-1]
plt.figure(figsize=(12,6))
plt.imshow(v)
plt.savefig('./img.png')
plt.close()
wandb.log({'Img':wandb.Image(cv2.imread('./img.png'))})
wandb.finish()

In [33]:
model = "COCO-Detection/faster_rcnn_R_50_C4_1x.yaml"

In [34]:
wandb.init(project=PROJECT_NAME,name='baseline')
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(model))
cfg.DATASETS.TRAIN = ('data')
cfg.DATASETS.TEST = ()
cfg.SOLVER.STEPS = []
cfg.SOLVER.MAX_ITER = 500
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(labels)
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(model)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()
predictor = DefaultPredictor(cfg)
cfg.MODEL.WEIGHTS = './output/model_final.pth'
evaluator = COCOEvaluator('data',output_dir='./output/')
val_loader = build_detection_test_loader(cfg,'data')
metrics = inference_on_dataset(predictor.model,val_loader,evaluator)
wandb.log(metrics)
img = cv2.imread('./data/vid_4_12220.jpg')
preds = predictor(img)
v = Visualizer(img[:,:,::-1],metadata=metadata)
v = v.draw_instance_predictions(preds['instances'].to('cpu'))
v = v.get_image()[:,:,::-1]
plt.figure(figsize=(12,6))
plt.imshow(v)
plt.savefig('./img.png')
plt.close()
wandb.log({'Img':wandb.Image(cv2.imread('./img.png'))})
wandb.finish()